In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder


# Load Data

In [2]:
url='https://docs.google.com/spreadsheets/d/e/2PACX-1vTbg5WVW6W3c8SPNUGc3A3AL-AG32TPEQGpdzARfNICMsLFI0LQj0jporhsLCeVhkN5AoRsTkn08AYl/pub?output=csv'
raw = pd.read_csv(url)

In [3]:
# Copy data
df =  raw.copy()
df.head()

,TransactionID,AccountID,TransactionAmount,PreviousTransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70.0,Doctor,81.0,1.0,5112.21,2024-11-04 8:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68.0,Doctor,141.0,1.0,13758.91,2024-11-04 8:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19.0,Student,56.0,1.0,1122.35,2024-11-04 8:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26.0,Student,25.0,1.0,8569.06,2024-11-04 8:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,NaN,Student,198.0,1.0,7429.40,2024-11-04 8:06:39


# Missing and Duplicated Value Cheking

In [4]:
# Missing values checking
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
TransactionID              29
AccountID                  21
TransactionAmount          26
PreviousTransactionDate    28
TransactionType            30
Location                   30
DeviceID                   30
IP Address                 20
MerchantID                 23
Channel                    27
CustomerAge                18
CustomerOccupation         23
TransactionDuration        26
LoginAttempts              21
AccountBalance             27
TransactionDate            24
dtype: int64


In [5]:
# Duplicated values checking
print("Jumlah baris duplikat:")
print(df.duplicated().sum())

Jumlah baris duplikat:
21


# Feature Scaling

In [6]:
# Parse date time
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'], errors='coerce')

In [7]:
# Create time diff feature
df['TimeSincePrevSeconds'] = (df['TransactionDate'] - df['PreviousTransactionDate']).dt.total_seconds()

In [8]:
# Create total transaction/account feature
df['TransactionCountTotal'] = df.groupby('AccountID')['TransactionID'].transform('count')

In [9]:
# Result Checking
print("Perubahan dan Penambahan Fitur: \n")
display(df[['TransactionDate', 'PreviousTransactionDate', 'TimeSincePrevSeconds', 'TransactionCountTotal']].head())

Perubahan dan Penambahan Fitur: 



,TransactionDate,PreviousTransactionDate,TimeSincePrevSeconds,TransactionCountTotal
0,2024-11-04 08:08:08,2023-04-11 16:29:14,49477134.0,7.0
1,2024-11-04 08:09:35,2023-06-27 16:44:19,42823516.0,7.0
2,2024-11-04 08:07:04,2023-07-10 18:16:08,41694656.0,5.0
3,2024-11-04 08:09:06,2023-05-05 16:32:11,47403415.0,8.0
4,2024-11-04 08:06:39,2023-10-16 17:51:24,33228915.0,6.0


In [10]:
# Numeric features
numeric_features = ['TransactionAmount', 'AccountBalance', 'TransactionDuration',
                    'CustomerAge', 'LoginAttempts', 'TimeSincePrevSeconds', 'TransactionCountTotal']

numeric_features = [col for col in numeric_features if col in df.columns]

In [11]:
df.head()

,TransactionID,AccountID,TransactionAmount,PreviousTransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionDate,TimeSincePrevSeconds,TransactionCountTotal
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70.0,Doctor,81.0,1.0,5112.21,2024-11-04 08:08:08,49477134.0,7.0
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68.0,Doctor,141.0,1.0,13758.91,2024-11-04 08:09:35,42823516.0,7.0
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19.0,Student,56.0,1.0,1122.35,2024-11-04 08:07:04,41694656.0,5.0
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26.0,Student,25.0,1.0,8569.06,2024-11-04 08:09:06,47403415.0,8.0
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,NaN,Student,198.0,1.0,7429.40,2024-11-04 08:06:39,33228915.0,6.0


In [12]:
# Copy df and create scaler
df_numeric = df[numeric_features].copy()
scaler = MinMaxScaler()

In [13]:
# Apply scaler to numeric features
scaled_arr = scaler.fit_transform(df_numeric.fillna(df_numeric.median()))
df_scaled = pd.DataFrame(scaled_arr, columns=numeric_features, index=df.index)

In [14]:
# Scaling Result
df[numeric_features] = df_scaled[numeric_features]
print("Head setelah scaling (numeric features):")
display(df.head())

Head setelah scaling (numeric features):


,TransactionID,AccountID,TransactionAmount,PreviousTransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionDate,TimeSincePrevSeconds,TransactionCountTotal
0,TX000001,AC00128,0.007207,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,0.838710,Doctor,0.244828,0.0,0.336832,2024-11-04 08:08:08,0.728036,0.545455
1,TX000002,AC00455,0.195940,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,0.806452,Doctor,0.451724,0.0,0.918055,2024-11-04 08:09:35,0.516531,0.545455
2,TX000003,AC00019,0.065680,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,0.016129,Student,0.158621,0.0,0.068637,2024-11-04 08:07:04,0.480647,0.363636
3,TX000004,AC00070,0.096016,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,0.129032,Student,0.051724,0.0,0.569198,2024-11-04 08:09:06,0.662117,0.636364
4,TX000005,AC00411,0.006874,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,0.435484,Student,0.648276,0.0,0.492591,2024-11-04 08:06:39,0.211537,0.454545


# Drop ID Column

In [15]:
# Melakukan drop pada kolom yang memiliki keterangan id dan IP Address
cols_to_drop = ['TransactionID', 'AccountID', 'DeviceID', 'IP Address', 'MerchantID']
cols_present_to_drop = [col for col in cols_to_drop if col in df.columns]
df = df.drop(columns=cols_present_to_drop)
print("Dropped columns:", cols_present_to_drop)

Dropped columns: ['TransactionID', 'AccountID', 'DeviceID', 'IP Address', 'MerchantID']


# Features Encoding for Categorical Features

In [17]:
# Caegorical feature for feature encoding
categorical_features = ['TransactionType', 'Location', 'Channel', 'CustomerOccupation']
categorical_features = [col for col in categorical_features if col in df.columns]

In [18]:
# Encode process
label_encoders = {}
for col in categorical_features:
  le = LabelEncoder()
  df[col] = le.fit_transform(df[col].fillna('_MISSING_').astype(str))
  label_encoders[col] = le

In [19]:
# Result checking
print("Head setelah encoding kategori:")
display(df.head())

Head setelah encoding kategori:


,TransactionAmount,PreviousTransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionDate,TimeSincePrevSeconds,TransactionCountTotal
0,0.007207,2023-04-11 16:29:14,1,36,0,0.838710,0,0.244828,0.0,0.336832,2024-11-04 08:08:08,0.728036,0.545455
1,0.195940,2023-06-27 16:44:19,1,15,0,0.806452,0,0.451724,0.0,0.918055,2024-11-04 08:09:35,0.516531,0.545455
2,0.065680,2023-07-10 18:16:08,1,23,2,0.016129,3,0.158621,0.0,0.068637,2024-11-04 08:07:04,0.480647,0.363636
3,0.096016,2023-05-05 16:32:11,1,33,2,0.129032,3,0.051724,0.0,0.569198,2024-11-04 08:09:06,0.662117,0.636364
4,0.006874,2023-10-16 17:51:24,0,1,2,0.435484,3,0.648276,0.0,0.492591,2024-11-04 08:06:39,0.211537,0.454545


In [21]:
# Features list checking
df.columns.tolist()

['TransactionAmount',
 'PreviousTransactionDate',
 'TransactionType',
 'Location',
 'Channel',
 'CustomerAge',
 'CustomerOccupation',
 'TransactionDuration',
 'LoginAttempts',
 'AccountBalance',
 'TransactionDate',
 'TimeSincePrevSeconds',
 'TransactionCountTotal']

# Handling Missing Values

In [22]:
for col in df.columns:
  # fillna for numeric
  if df[col].dtype.kind in 'biufc':
    df[col] = df[col].fillna(df[col].median())
  # fillna for categorical
  else:
    df[col] = df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else '_MISSING_')

In [23]:
display(df.head())

,TransactionAmount,PreviousTransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionDate,TimeSincePrevSeconds,TransactionCountTotal
0,0.007207,2023-04-11 16:29:14,1,36,0,0.838710,0,0.244828,0.0,0.336832,2024-11-04 08:08:08,0.728036,0.545455
1,0.195940,2023-06-27 16:44:19,1,15,0,0.806452,0,0.451724,0.0,0.918055,2024-11-04 08:09:35,0.516531,0.545455
2,0.065680,2023-07-10 18:16:08,1,23,2,0.016129,3,0.158621,0.0,0.068637,2024-11-04 08:07:04,0.480647,0.363636
3,0.096016,2023-05-05 16:32:11,1,33,2,0.129032,3,0.051724,0.0,0.569198,2024-11-04 08:09:06,0.662117,0.636364
4,0.006874,2023-10-16 17:51:24,0,1,2,0.435484,3,0.648276,0.0,0.492591,2024-11-04 08:06:39,0.211537,0.454545


# Handling Duplicated Value

In [25]:
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"Jumlah baris duplikat yang dihapus: {before - after}")
print(f"Jumlah baris setelah penghapusan duplikat: {after}")

Jumlah baris duplikat yang dihapus: 23
Jumlah baris setelah penghapusan duplikat: 2514


In [26]:
display(df.head())

,TransactionAmount,PreviousTransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionDate,TimeSincePrevSeconds,TransactionCountTotal
0,0.007207,2023-04-11 16:29:14,1,36,0,0.838710,0,0.244828,0.0,0.336832,2024-11-04 08:08:08,0.728036,0.545455
1,0.195940,2023-06-27 16:44:19,1,15,0,0.806452,0,0.451724,0.0,0.918055,2024-11-04 08:09:35,0.516531,0.545455
2,0.065680,2023-07-10 18:16:08,1,23,2,0.016129,3,0.158621,0.0,0.068637,2024-11-04 08:07:04,0.480647,0.363636
3,0.096016,2023-05-05 16:32:11,1,33,2,0.129032,3,0.051724,0.0,0.569198,2024-11-04 08:09:06,0.662117,0.636364
4,0.006874,2023-10-16 17:51:24,0,1,2,0.435484,3,0.648276,0.0,0.492591,2024-11-04 08:06:39,0.211537,0.454545


# Handling Outlier

In [28]:
def quantile_winsorize_series(s, lower_q=0.01, upper_q=0.99):
    lower = s.quantile(lower_q)
    upper = s.quantile(upper_q)
    return s.clip(lower, upper)


In [29]:
numeric_features = [col for col in numeric_features if col in df.columns]

for col in numeric_features:
    df[col] = quantile_winsorize_series(df[col])

In [39]:
# Result checking
print("Hasil setelah handling outlier:")
display(df[numeric_features].describe())

Hasil setelah handling outlier:


,TransactionAmount,AccountBalance,TransactionDuration,CustomerAge,LoginAttempts,TimeSincePrevSeconds,TransactionCountTotal
count,2514.000000,2514.000000,2514.000000,2514.000000,2514.000000,2514.000000,2514.000000
mean,0.153717,0.336688,0.377439,0.429306,0.030430,0.494903,0.460331
std,0.147835,0.260169,0.239787,0.285942,0.148674,0.289544,0.201938
min,0.002158,0.004353,0.010345,0.000000,0.000000,0.011165,0.090909
25%,0.043213,0.095324,0.182759,0.145161,0.000000,0.244710,0.272727
50%,0.110014,0.311416,0.351724,0.435484,0.000000,0.489080,0.454545
75%,0.213328,0.506870,0.520690,0.661290,0.000000,0.749985,0.636364
max,0.708927,0.966017,0.971966,0.983871,1.000000,0.997033,1.000000


# Binning Data

In [42]:
bin_cols = []
if 'TransactionAmount' in df.columns:
  bin_cols.append('TransactionAmount')
if 'LoginAttempts' in df.columns:
  bin_cols.append('LoginAttempts')

In [44]:
# Binning Transaction Ammount
if 'TransactionAmount' in bin_cols:
  df['TransactionAmountBin'] = pd.cut(
      df['TransactionAmount'],
      bins=[
            -1,
            df['TransactionAmount'].quantile(0.5),
            df['TransactionAmount'].quantile(0.75),
            df['TransactionAmount'].quantile(0.9),
            1.0
          ],
      labels=['Low', 'Medium', 'High', 'Very High']
  )

  le_amt = LabelEncoder()
  df['TransactionAmountBin'] = le_amt.fit_transform(df['TransactionAmountBin'].astype(str))


In [46]:
# Binning Login Attempts
if 'LoginAttempts' in bin_cols:
  df['LoginAttemptsBin'] = pd.cut(
      df['LoginAttempts'],
      bins=[-1, 0.2, 0.6, 1.0],
      labels=['Normal', 'Suspicious', 'HighRisk']
  )
  le_login = LabelEncoder()
  df['LoginAttemptsBin'] = le_login.fit_transform(df['LoginAttemptsBin'].astype(str))


In [48]:
# Result checking
print("Hasil setelah binning:")
display(df[[col for col in df.columns if 'Bin' in col or col in bin_cols]].head())


Hasil setelah binning:


,TransactionAmount,LoginAttempts,TransactionAmountBin,LoginAttemptsBin
0,0.007207,0.0,1,1
1,0.195940,0.0,2,1
2,0.065680,0.0,1,1
3,0.096016,0.0,1,1
4,0.006874,0.0,1,1


In [50]:
df.describe()

,TransactionAmount,PreviousTransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionDate,TimeSincePrevSeconds,TransactionCountTotal,TransactionAmountBin,LoginAttemptsBin
count,2514.000000,2514,2514.000000,2514.000000,2514.00000,2514.000000,2514.000000,2514.000000,2514.000000,2514.000000,2514,2514.000000,2514.000000,2514.000000,2514.000000
mean,0.153717,2023-07-03 18:14:20.636435968,0.787987,21.284010,1.01074,0.429306,1.530231,0.377439,0.030430,0.336688,2024-11-04 08:09:22.405330176,0.494903,0.460331,1.294749,0.998409
min,0.002158,2023-01-02 16:00:06,0.000000,0.000000,0.00000,0.000000,0.000000,0.010345,0.000000,0.004353,2024-11-04 08:06:23,0.011165,0.090909,0.000000,0.000000
25%,0.043213,2023-03-29 16:26:04.249999872,1.000000,11.000000,0.00000,0.145161,0.250000,0.182759,0.000000,0.095324,2024-11-04 08:07:54.249999872,0.244710,0.272727,1.000000,1.000000
50%,0.110014,2023-07-06 04:57:02.500000,1.000000,21.000000,1.00000,0.435484,2.000000,0.351724,0.000000,0.311416,2024-11-04 08:09:21,0.489080,0.454545,1.000000,1.000000
75%,0.213328,2023-10-05 16:41:45,1.000000,32.000000,2.00000,0.661290,3.000000,0.520690,0.000000,0.506870,2024-11-04 08:10:53,0.749985,0.636364,2.000000,1.000000
max,0.708927,2024-01-01 18:21:50,2.000000,43.000000,3.00000,0.983871,4.000000,0.971966,1.000000,0.966017,2024-11-04 08:12:23,0.997033,1.000000,3.000000,2.000000
std,0.147835,NaN,0.437042,12.397534,0.83084,0.285942,1.150477,0.239787,0.148674,0.260169,NaN,0.289544,0.201938,0.841755,0.218516


# Save df for Model Clustering

In [37]:
df.to_parquet('data.parquet', index=False)